#### 前言

在 Machine Learning 中，我們經常需要定義(Define)、儲存(Save)或是回復(Restore)模型。模型其實就是一個 Function，且 Function 裡頭的參數能夠在 Training 時被更新。

本篇在於了解 Keras 底下的 TensorFlow 是如何運作的！

In [1]:
import tensorflow as tf

#### 了解 Model 與 Layer

大部分的 Model 都是由許多 Layer 所組成。我們可以將每一層 Layer 視為一組數學運算（例如：矩陣運算），Layer 中也包含許多可以訓練的參數。Keras 中提供許多 High-Level 的 API 來實作 Layer 與 Model，其實他們的底層是 TensorFlow Module

In [2]:
# a simple tensorflow module
class SimpleModule(tf.Module):
    def __init__(self, name=None):
        super().__init__(name=name)
        self.a_variable = tf.Variable(5.0, name="train_me")
        self.a_non_trainable_variable = tf.Variable(5.0, trainable=False, name="do_not_train_me")
    def __call__(self, x):
        return self.a_variable * x + self.a_non_trainable_variable

In [3]:
simple_module = SimpleModule(name="simple")
simple_module(tf.constant(5.0))

<tf.Tensor: shape=(), dtype=float32, numpy=30.0>

不管是 Module 還是基於它的 Layer 和 Model，他們都是 Python Object。因此，他們都會有 Internal State (Property)，以及使用那些 State 的 Method。

執得注意的是，當一個 SimpleModule Class 繼承 TensorFlow Module 後，SimpleModule Object 中的 Property 如果是 TensorFlow Variable 或是 TensorFlow Module 都會自動被「收集」。因此，能夠輕易的儲存、載入、更新這些 Variable。

In [4]:
# all trainable variable
print(simple_module.trainable_variables)

# all variable
print(simple_module.variables)

(<tf.Variable 'train_me:0' shape=() dtype=float32, numpy=5.0>,)
(<tf.Variable 'do_not_train_me:0' shape=() dtype=float32, numpy=5.0>, <tf.Variable 'train_me:0' shape=() dtype=float32, numpy=5.0>)


2022-01-27 20:28:35.755971: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


#### 建立一個 Two Layer (Linear Layer) Model

In [5]:
# 先定義 Dense Layer
class Dense(tf.Module):
    def __init__(self, in_feature, out_feature, name=None):
        super().__init__(name=name)
        self.w = tf.Variable(tf.random.normal([in_feature, out_feature]), name='w')
        self.b = tf.Variable(tf.zeros([out_feature]), name='b')
    def __call__(self, x):
        y = tf.matmul(x, self.w) + self.b
        return tf.nn.relu(y)

In [6]:
# 再定義一個 Model
class SequentialModel(tf.Module):
    def __init__(self, name=None):
        super().__init__(name=name)
        
        self.dense_1 = Dense(in_feature=3, out_feature=3)
        self.dense_2 = Dense(in_feature=3, out_feature=2)

    def __call__(self, x):
        x = self.dense_1(x)
        return self.dense_2(x)

In [7]:
my_model = SequentialModel()
print(my_model(tf.constant([[2.0, 2.0, 2.0]])))

tf.Tensor([[0.       5.364657]], shape=(1, 2), dtype=float32)


因為 SequentialModel Object 是一個 TensorFlow Module，因此他會自動「收集」TensorFlow Module or TensorFlow Variable Property。使得我們可以簡單的儲存、使用一個 Model 中的所有 TensorFlow Module 與 TensorFlow Variable

In [8]:
my_model.submodules

(<__main__.Dense at 0x12645cc10>, <__main__.Dense at 0x105f43700>)

In [9]:
my_model.variables

(<tf.Variable 'b:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)>,
 <tf.Variable 'w:0' shape=(3, 3) dtype=float32, numpy=
 array([[ 0.2657629 , -0.7628884 ,  1.3672371 ],
        [-0.53525907,  0.91181624,  0.33850807],
        [-0.59865075, -1.0534652 , -0.21244623]], dtype=float32)>,
 <tf.Variable 'b:0' shape=(2,) dtype=float32, numpy=array([0., 0.], dtype=float32)>,
 <tf.Variable 'w:0' shape=(3, 2) dtype=float32, numpy=
 array([[-2.2294002, -0.7171399],
        [ 0.941205 , -1.5262581],
        [-0.8303009,  1.7962433]], dtype=float32)>)

如果使用過 tf.keras.layers.Dense 就可以發現，在建立 Dense 時，可以不用指定 Input Size。為了做到這樣，我們可以等到有東西傳入 Dense 時，再建立 Dense 裡面的 w 與 b。

In [10]:
class FlexibleDense(tf.Module):
    # 不需要指定 in_feature
    def __init__(self, out_features, name=None):
        super().__init__(name=name)
        self.is_built = False
        self.out_features = out_features

    def __call__(self, x):
        if not self.is_built:
            self.w = tf.Variable(tf.random.normal([x.shape[-1], self.out_features]), name='w')
            self.b = tf.zeros([self.out_features], name='b')
            self.is_built = True
        y = tf.matmul(x, self.w) + self.b
        return tf.nn.relu(y)

In [11]:
class MySequentialModel(tf.Module):
    def __init__(self, name=None):
        super().__init__(name=name)
        self.dense_1 = FlexibleDense(out_features=3)
        self.dense_2 = FlexibleDense(out_features=2)
    
    def __call__(self, x):
        x = self.dense_1(x)
        return self.dense_2(x)

In [12]:
my_model = MySequentialModel()
print(my_model(tf.constant([[2.0, 2.0, 2.0]])))

tf.Tensor([[5.1857915 3.1063278]], shape=(1, 2), dtype=float32)


#### 儲存 Model Weight

通常我們說儲存一個 TensorFlow Model，經常指的是 (1) 將 TensorFlow Module 儲存為 Checkpoint 或是 (2) 儲存為SavedModel。

Checkpoint 僅有紀錄 Module 與 Submodule 中的所有 TensorFlow Variable 的數值。當我們在使用 Checkpoint 時，Source Code 中還必須要有 Model 的結構，並載入這個 Chekcpoint，這個 Checkpoint 才有用。

SavedModel 則會將 Model(TensorFlow Module) 中的所有運算流程都存下來，當然也包含 Checkpoint。當我們在使用 SavedModel 時，其實不用管 Source Code 中如何定義這個 Model。因此，SavedModel 很適合作為 Deployment 的工具。

In [13]:
chkp_path = 'checkpoints/my_checkpoint'
checkpoint = tf.train.Checkpoint(model=my_model)
checkpoint.write(chkp_path)

'checkpoints/my_checkpoint'

一份 Checkpoint 中會包含兩個檔案：Data 與 Metadata。Data 就是所有 TensorFlow Variable 的數值；Metadata 則是紀錄總共有多少個 Checkpoint。

In [14]:
# 看看一份 checkpoint 中記錄了什麼 variable
tf.train.list_variables(chkp_path)

[('_CHECKPOINTABLE_OBJECT_GRAPH', []),
 ('model/dense_1/w/.ATTRIBUTES/VARIABLE_VALUE', [3, 3]),
 ('model/dense_2/w/.ATTRIBUTES/VARIABLE_VALUE', [3, 2])]

In [15]:
# 將 checkpoint restore 回去 model
my_model = MySequentialModel()
new_checkpoint = tf.train.Checkpoint(model=my_model)
new_checkpoint.restore(chkp_path)

In [16]:
my_model(tf.constant([[2.0, 2.0, 2.0]]))

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[5.1857915, 3.1063278]], dtype=float32)>

如果要將模型進行部署與分享，比較好的儲存模型的方法是透過 SavedModel。

`tf.saved_model.save()`
會建立一個資料夾，裡頭的 variables 資料夾代表 Model 的 Checkpoint (Model 中所有 Variable 的數值)；pb 檔案則是一個 protocol buffer 用來描述 Model 的 TensorFlow Graph。

In [17]:
tf.saved_model.save(my_model, "my_saved_model")

INFO:tensorflow:Assets written to: my_saved_model/assets


直接透過 `tf.saved_model.load()` 即可將剛剛儲存的模型載入！

In [18]:
new_model = tf.saved_model.load('my_saved_model')

In [19]:
# 但是新載入的模型是 TensorFlow User Object 而不是原先的 MySequentialModel Object
print(f'type: {type(new_model)}')
isinstance(new_model, MySequentialModel)

type: <class 'tensorflow.python.saved_model.load.Loader._recreate_base_user_object.<locals>._UserObject'>


False

#### 瞭解 Keras Model 與 Layer

Keras 中的 Layer 與 Model (High Level API) 最底層皆是繼承自 TensorFlow Module。舉例來說，`tf.keras.layers.Layer` 是 Keras 所有 layers 的 base class，其就繼承自 `tf.Module` (TensorFlow Module)

我們可以簡單的建立一個 Keras Layer，只需要將原來繼承自 `tf.Module` 改為 `tf.keras.layers.Layer`，並將 `__call__` 更改為 `call`。因為 Keras Layer 會在 `__call__` 中實作一些功能再呼叫 `call`

In [20]:
class MyDense(tf.keras.layers.Layer):

    def __init__(self, in_features, out_features, **kwargs):
        super().__init__(**kwargs)
        self.w = tf.Variable(tf.random.normal([in_features, out_features]), name='w')
        self.b = tf.Variable(tf.zeros([out_features]), name='b')

    def call(self, x):
        y = tf.matmul(x, self.w) + self.b
        return tf.nn.relu(y)

In [21]:
simple_layer = MyDense(name='simple', in_features=3, out_features=3)

In [22]:
simple_layer([
    [2.0, 2.0, 2.0]
])

<tf.Tensor: shape=(1, 3), dtype=float32, numpy=array([[0., 0., 0.]], dtype=float32)>

通常我們會希望確定 Model 的 Input Sahpe 之後，再建立 Model 中的 Weight。此時，我們可以透過 Keras Layer (`tf.keras.layers.Layer`) 中的 `build()` Method 做到這件事情。Keras Layer 的 `build()` Method 只會被呼叫一次，而且被呼叫時必須提供 Input Data 的 Shape，因此我們通常會將 Model 的 Weight 的建立放在 `build()` 中。

In [23]:
class FlexibleDense(tf.keras.layers.Layer):

    # Keras Layer 的 Constructor 必須包含 '**kwargs'，因為 Keras Layer 還支援很多其他的參數
    def __init__(self, out_features, **kwargs):
        super().__init__(**kwargs)
        self.out_features = out_features

    # 在 Keras Layer 的 build() 中建立 Weight
    def build(self, input_shape):
        self.w = tf.Variable(tf.random.normal([input_shape[-1], self.out_features]), name='w')
        self.b = tf.Variable(tf.zeros([self.out_features]), name='b')

    # 在 Keras Layer 的 call() 中定義 Layer 的計算流程 (Computation)
    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b

In [24]:
flexible_dense = FlexibleDense(out_features=3)

In [26]:
# 因為 Keras Layer 還沒有被 'build' 因此 Weight 還沒有被建立
flexible_dense.variables

[]

In [29]:
# 使用 Keras Layer 時 (輸入資料到 Keras Layer 中)，Kera Layer 的 build() 就會被呼叫
input_tensor = tf.constant(
    [
        [2.0, 2.0, 2.0],
        [3.0, 3.0, 3.0]
    ]
)
flexible_dense(input_tensor)

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[-3.0745847 , -0.79397327, -0.48884273],
       [-4.611877  , -1.1909598 , -0.7332642 ]], dtype=float32)>

提醒：因為 Keras Layer 中的 `build()` 只會被呼叫一次，一旦呼叫過後，Keras Layer 中的 Weight 就被建立了，所能接受的 Input Shape 也會固定下來。

In [31]:
flexible_dense.variables

[<tf.Variable 'flexible_dense/w:0' shape=(3, 3) dtype=float32, numpy=
 array([[-0.1036355 ,  0.6960638 , -0.01906842],
        [-0.14984168, -1.0727694 ,  0.80770296],
        [-1.2838151 , -0.02028105, -1.0330559 ]], dtype=float32)>,
 <tf.Variable 'flexible_dense/b:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)>]

In [32]:
input_tensor = tf.constant(
    [
        [2.0, 2.0, 2.0, 2.0]
    ]
)

flexible_dense(input_tensor)

InvalidArgumentError: Matrix size-incompatible: In[0]: [1,4], In[1]: [3,3] [Op:MatMul]

#### 實作 Keras Model

我們可以透過繼承 `tf.keras.layers.Layer` 建立一個 Keras Layer；同樣的，我們可以透過繼承 `tf.keras.Model` 建立一個 Keras Model。

In [33]:
class MySequentialModel(tf.keras.Model):
    def __init__(self, name=None, **kwargs):
        super().__init__(**kwargs)
        self.dense_1 = FlexibleDense(out_features=3)
        self.dense_2 = FlexibleDense(out_features=2)

    # 將 Keras Model 的計算流程寫在 call() 中
    def call(self, inputs):
        x = self.dense_1(inputs)
        return self.dense_2(x)

In [34]:
my_sequential_model = MySequentialModel(name="seq_model")

In [35]:
input_tensor = tf.constant(
    [
        [2.0, 2.0, 2.0]
    ]
)

my_sequential_model(input_tensor)

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[ 12.812953, -12.223006]], dtype=float32)>

In [36]:
# 查看 Keras Model 中的 Variable (Weight)
my_sequential_model.variables

[<tf.Variable 'my_sequential_model/flexible_dense_1/w:0' shape=(3, 3) dtype=float32, numpy=
 array([[-1.3047113 ,  1.8306013 ,  0.41649497],
        [ 0.16001828, -1.0010756 , -0.39578095],
        [-2.7144563 ,  0.11053286,  1.9898789 ]], dtype=float32)>,
 <tf.Variable 'my_sequential_model/flexible_dense_1/b:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)>,
 <tf.Variable 'my_sequential_model/flexible_dense_2/w:0' shape=(3, 2) dtype=float32, numpy=
 array([[-1.0462555 ,  1.4136693 ],
        [ 1.6629859 ,  0.74568456],
        [ 0.40063584, -0.67489016]], dtype=float32)>,
 <tf.Variable 'my_sequential_model/flexible_dense_2/b:0' shape=(2,) dtype=float32, numpy=array([0., 0.], dtype=float32)>]

In [37]:
# 查看 Keras Model 中的 Submodule
my_sequential_model.submodules

(<__main__.FlexibleDense at 0x1264e2b80>,
 <__main__.FlexibleDense at 0x126c07b20>)